In [3]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_wrapper, .output {height:auto;}</style>"))

C:\Users\Ilham\AppData\Local\Temp\ipykernel_20744\3540611931.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine

pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [97]:
def scrape_items(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    items = []
    # scraping
    ItemDesc = soup.find_all('p', {'class': 'bl-text bl-text--body-14 bl-text--ellipsis__2'})
    for item in ItemDesc:
        items.append(item.text.strip())
    return items


In [98]:

baselink = 'https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat'
baselinkpg = 'https://www.bukalapak.com/products?page={}&search%5Bkeywords%5D=sepeda%20lipat'

# save all pages
all_items = []

for i in range(1, 100):
    if i == 1:
        link = baselink
    else:
        link = baselinkpg.format(i)
    
    page_content = fetch_page(link)
    
    if page_content:
        items = scrape_items(page_content)
        all_items.extend(items)


HTTP request ke https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=2&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=3&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=4&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=5&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=6&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=7&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=8&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=9&search%5Bkeywords%5D=sepeda%20lipat berhasil
HTTP request ke https://www.bukalapak.com/products?page=10&search%5Bkeywords%5D=

In [99]:
df = pd.DataFrame(all_items, columns=['ItemName'])
print(df)

                                                                                                                                                        ItemName
0                                                                                            Paket Kabel Shifter TRLREQ Set Sepeda Lipat MTB Roadbike Front Rear
1                                                               Sadel Jok Sepeda Empuk MTB Gunung Lipat Fixie Ontel Saddle Soft Hollow Cushion ROCKBROS 38210006
2                                                                     Element Sepeda Lipat 20 Inchi FoldX Lite Damn  I Love Indonesia Limited Edition New Design
3                                                                                             Sepeda Lipat Element Ecosmo 10 Speed LIMITED EDITION SUMPAH PEMUDA
4                                                                                                         Sepeda Lipat Pikes Gen 2 Special Edition Pevita Pearce
5                                 

In [92]:
baselink = 'https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat'
baselinkpg = 'https://www.bukalapak.com/products?page={}&search%5Bkeywords%5D=sepeda%20lipat'

listresponse = []
for i in range(1,4):
    if i == 1:
        link = baselink.format(i)
        try:
            response = requests.get(link)
            if response.status_code == 200:
                print('requests http terhadap link ' + str(i) + ' berhasil')
                print(link)
            else:
                print('requests http terhadap link ' + str(i) + ' berhasil tetapi perlu hak akses')
                print(link)
        except:
            print('requests http terhadap link ' + str(i) + ' tidak berhasil')
            print(link)
    else:
        linkpg = baselinkpg.format(i)
        try:
            response = requests.get(linkpg)
            if response.status_code == 200:
                print('requests http terhadap linkpg ' + str(i) + ' berhasil')
                print(linkpg)
            else:
                print('requests http terhadap linkpg ' + str(i) + ' berhasil tetapi perlu hak akses')
                print(linkpg)
        except:
            print('requests http terhadap linkpg ' + str(i) + ' tidak berhasil')
            print(linkpg)

if response.status_code == 200:
    ItemDesc = soup.find_all('p', {'class': 'bl-text bl-text--body-14 bl-text--ellipsis__2'})
    # ItemName.text.strip()
    items = []
    for item in ItemDesc:
        items.append(item.text.strip())

df = pd.DataFrame(items, columns=['ItemName'])

print(df)


requests http terhadap link 1 berhasil
https://www.bukalapak.com/products?search%5Bkeywords%5D=sepeda%20lipat
requests http terhadap linkpg 2 berhasil
https://www.bukalapak.com/products?page=2&search%5Bkeywords%5D=sepeda%20lipat
requests http terhadap linkpg 3 berhasil
https://www.bukalapak.com/products?page=3&search%5Bkeywords%5D=sepeda%20lipat
                                                                                                  ItemName
0                                              Seatpost ORI SEPEDA LIPAT ALLOY CNC Ultralight 33.9mmx600mm
1              Sadel Sepeda Super Empuk Kursi Jok MTB Gunung Lipat Saddle Handle Bike Road Ventilasi Udara
2                                      Xiaomi HIMO Z20 Sepeda Lipat Elektrik Smart Moped Bicycle 250W 80KM
3               Sepeda Lankeleisi G650 Elektrik Lipat Smart Moped Deluxe Version 48V 10.4Ah - Black-Yellow
4                                             Sepeda Lipat Exotic Second Sepeda Lipat Bekas United Pasific
5         

In [9]:
# Define connection
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-B14RR7Q, 1433;" # ConnectionName and port
    "DATABASE=Abstract;"
    "UID=sa;"
    "PWD=123;"
)

# Create a connection
conn = pyodbc.connect(conn_str)


In [27]:
table_name = 'Province'

for index, row in df.iterrows():
    sql = f"INSERT INTO {table_name} (No, Provinsi, Kode, ID, Kep, Ibu_Kota, Wilayah_Geografis, Gubernur, Hari_Jadi, UU, Kab, Kota) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    conn.execute(sql, row['No'], row['Provinsi'], row['Kode'], row['ID'], row['Kep.'], row['Ibu kota'], row['Wilayah geografis'], row['Gubernur'], row['Hari jadi'], row['UU'], row['Kab'], row['Kota'])

conn.commit()

conn.close()


In [35]:
## Insert langsung ke table nya tanpa harus create dulu
server = 'DESKTOP-B14RR7Q,1433'
database = 'Abstract'
username = 'sa'
password = '123'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

engine = create_engine(connection_string)

table_name = 'Test'
df.to_sql(table_name, con=engine, if_exists='append', index=False)

print("Data inserted successfully!")


Data inserted successfully!
